## Errors-in-variables estimation of specific climate sensitivity
by D. Heslop (david.heslop@anu.edu.au), E.J. Rohling, G. L. Foster, & J. Yu, submitted.

### Notebook 2 - Perform EIV regression
Perform EIV regression using the time series created in Notebook 1

#### Import packages
Package requirements are provided in the accompanying ```requirements.txt``` file.

N.B., for this example notebook needs to be run twice, once for ```CO2_2ppm.txt``` and once for ```CO2_20ppm.txt```.

In [1]:
import numpy as np
from scipy.optimize import minimize

#### Define the files containing the regression variables

In [2]:
xfile = 'CO2_20ppm.txt' #define the binned CO2 record with 2ppm uncertainties
yfile = 'GMST_bin.txt' #define the binned GMST record file

#### Set the random number generator seed to ensure repeatability

In [3]:
seed_value = 54321 #random number generator seed
np.random.seed(seed_value) #set the random number generator

#### Define the CO$_2$ radiative forcing effect 

A first-order approximation of the radiative forcing effect, $\Delta R_{\mathrm{[CO2]}}$, of CO$_2$ is (Myhre, 1998):

\begin{equation}
f_{\Delta R\mathrm{[CO2]}}(\mathrm{CO}_2) = 5.35 \log(\mathrm{CO}_2 / C_0),
\end{equation}

where $C_0$ is a reference CO$_2$ concentration. Conversion of CO$_2$ into radiative forcing is defined in the ```DeltaF_CO2``` function. 

In [4]:
def DeltaF_CO2(CO2,C0):
    
    ## INPUTS
    # CO2 = CO2 concentration
    # C0 = reference CO2 concentration

    ## OUTPUT
    # radiative forcing in W/m2
    
    return 5.35*np.log((CO2)/C0)

#### Define EIV objective function based on the negative log-likelihood (manuscript equ 23)
In this case the negative log-likelihood is defined by two terms:

```dy``` = Gaussian uncertainty on the GMST with an additional Gaussian scatter term

```dx``` = Uncertainty on the radiative forcing defined by the distribution in Appendix A



In [5]:
def nll(X,x,y,dx,dy,C0,counts): #funtion to find negative log-likelihood
    
    ## INPUTS
    # X = array containing [true CO2 values, sigma, gradient of fit, intercept of fit]
    # x = observed CO2 values
    # y = observed GMST values
    # dx = standard deviation of CO2 uncertainty
    # dy = standard deviation of GMST uncertainty
    # C0 = reference CO2 concentration
    # counts - how many times each point should be considered (used in bootstrapping)

    ## OUTPUT
    # negative log-likelihood of data given the model
    
    n = np.size(x) #number of data points
    gamma = DeltaF_CO2(X[:n],C0) #estimated true CO2 values converted to radiative forcing
    sigma = np.abs(X[n]) #standard deviation of scatter in y-variable
    pp = X[n+1:] #polynomial fit
    
    mu = np.polyval(pp,gamma) #predictions of true y-values
    tau = np.sqrt(sigma**2+dy**2) #combine y-errors with scatter defined by sigma
    
    #negative log-likelihood for y variable (assuming Gaussian)
    term1 = 0.5*((y-mu)/tau)**2
    term1 += np.log(tau*np.sqrt(2*np.pi))
    
    #negative log-likeihood for x variable (CO2 radiative forcing)
    term2 = 0.5*((C0*np.exp(gamma/5.35)-x)/dx)**2
    term2 += np.log(dx*np.sqrt(2*np.pi))
    term2 -= np.abs(gamma/5.35+np.log(C0)-np.log(5.35))
    
    return np.sum((term1+term2)*counts) #negative log-likelihood

#### EIV uncertainties are estimated via block bootstrapping (required because we are working with time series rather than independent data)

Function selects a collection of blocks with length defined by Hall (1995, doi:10.2307/2337534).

The function needs to be run for each bootstrap iteration.

In [6]:
def block_bootstrap(n):
    
    ## INPUTS
    # n = number of points in the time series
    
    ## OUTPUTS
    # block_idx = indicies of all points selected by the bootstrap
    # unique_values = unique indicies
    # counts = number of times each of the unique indicies occurs within the bootstrap
    
    lb = int(np.ceil(n**(1/5))) #block length
    Nb = int(np.floor(n/lb)) #number of blocks to include   

    blocks = np.zeros((Nb,lb)) #preallocate array to store block indicies
    blocks[:,0] = np.random.randint(low=0,high=n-1,size=Nb) #randomly choose starting point of each block
    for i in range(1,lb): #loop to add subsequent points to each block
        blocks[:,i] = blocks[:,i-1]+1
    blocks = np.concatenate(np.mod(blocks,n)) #periodic boundaries to stay within record limits
    unique_values, counts = np.unique(blocks, return_counts=True) #unique indicies and occurance counts

    return blocks.astype(int), unique_values.astype(int), counts

#### Load and define the variables needed for the regression (see file name definitions above)

In [7]:
x = np.loadtxt(xfile)[:,0] #x-variable bin means
dx = np.loadtxt(xfile)[:,1] #x-variable bin standard deviations
n = np.size(x) #number of bins to be considered
C0 = x[0] #reference CO2 concentration

y = np.loadtxt(yfile)[:,0] #y-variable bin means
dy = np.loadtxt(yfile)[:,1] #y-variable bin standard deviations

#### Perform the block bootstrap for the EIV, OLSi, and OLSo approaches (this can take a little time) 

```OLSi``` = Ordinary least-squares regression including an intercept term

```OLSo``` = Ordinary least-squares regression without an intercept term

In [8]:
niter = 1000 # number of block boostrap iterations

#Define some model parameters
order = 1 #EIV fit is 1st order polynomial
options={'maxiter': int(10000*n)} #minimization options
sigma0 = 0.5 #initial estimate for sigma

pp_EIV = np.zeros((niter,order+1)) #preallocate array for EIV regression coefficients
pp_OLSi = np.zeros((niter,order+1)) #preallocate array for OLSi regression coefficients
pp_OLSo = np.zeros((niter,order)) #preallocate array for OLSo regression coefficients

D = np.transpose(np.vstack((DeltaF_CO2(x,C0),np.ones(np.size(x))))) #OLSi and OLSo regression design matrix

i = 0 #set counter to track completed bootstrap iterations
print("Running.....")
while i<niter:
    blocks, idx, counts = block_bootstrap(n) #select bootstrap indicies
    
    #form EIV optimization initialization
    ppy = np.polyfit(DeltaF_CO2(x[idx],C0),y[idx],1) # regress y on x
    ppx = np.polyfit(y[idx],DeltaF_CO2(x[idx],C0),1) # regress x on y
    y0 = np.polyval(ppy,DeltaF_CO2(x[idx],C0))
    y1 = (DeltaF_CO2(x[idx],C0)-ppx[1])/ppx[0]
    pp = np.polyfit(DeltaF_CO2(x[idx],C0),0.5*(y0+y1),1) #average of y on x and x on y regressions 
    
    Xini = np.hstack((x[idx],sigma0,pp)) #form initialization array
    Fbb = minimize(nll, Xini, args=(x[idx],y[idx],dx[idx],dy[idx],C0,counts),options=options) #minimize nll
    pp_EIV[i,:] = Fbb.x[-order-1:] #record solution
    
    pp_OLSi[i,:] = np.dot(np.linalg.pinv(D[blocks,:]),y[blocks]) #perform OLSi regression
    pp_OLSo[i] = np.dot(np.linalg.pinv(D[blocks,0:1]),y[blocks]) #perform OLSo regression
    
    
    if Fbb.success: #only accept result if the minimize function converged
        i += 1
        if np.mod(i,50)==0:
            print("Completed iteration "+format(i)+" of "+format(niter))

Running.....
Completed iteration 50 of 1000
Completed iteration 100 of 1000
Completed iteration 150 of 1000
Completed iteration 200 of 1000
Completed iteration 250 of 1000
Completed iteration 300 of 1000
Completed iteration 350 of 1000
Completed iteration 400 of 1000
Completed iteration 450 of 1000
Completed iteration 500 of 1000
Completed iteration 550 of 1000
Completed iteration 600 of 1000
Completed iteration 650 of 1000
Completed iteration 700 of 1000
Completed iteration 750 of 1000
Completed iteration 800 of 1000
Completed iteration 850 of 1000
Completed iteration 900 of 1000
Completed iteration 950 of 1000
Completed iteration 1000 of 1000


#### Output the results in preperation for plotting

In [9]:
if xfile == 'CO2_20ppm.txt':
    np.savetxt("CO2_20ppm_OLSo.txt",pp_OLSo) #save the OLSo regression coefficients
    np.savetxt("CO2_20ppm_OLSi.txt",pp_OLSi) #save the OLSi regression coefficients
    np.savetxt("CO2_20ppm_EIV.txt",pp_EIV) #save the EIV regression coefficients
else:
    np.savetxt("CO2_2ppm_OLSo.txt",pp_OLSo) #save the OLSo regression coefficients
    np.savetxt("CO2_2ppm_OLSi.txt",pp_OLSi) #save the OLSi regression coefficients
    np.savetxt("CO2_2ppm_EIV.txt",pp_EIV) #save the EIV regression coefficients        